In [1]:
!pip install numpy pandas
!pip install openpyxl #attribute to install excel reading
%pip install matplotlib seaborn plotly seaborn yfinance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from pandas import to_datetime




def rename_column(df, str_column_name, str_new_name):
    time_periods = ['1W', '1M', '1Q', '1Y', '2Y', '5Y', '10Y', '15Y', '20Y']
    
    # Match suffix and assign new name accordingly
    for i in range(1, 9):
        if f'.{i}' in str_column_name:
            new_col_name = str_new_name + '_' + time_periods[i]
            df.rename(columns={str_column_name: new_col_name}, inplace=True)
            return df
            

    # If no .1 to .8 is found, assume it's the first (base) column
    df.rename(columns={str_column_name: str_new_name + '_' + time_periods[0]}, inplace=True)
    return df





# function for calling the rename functions across all the swaps
def call_rename(df, str_swap):
    list_space = ['' , 1,2,3,4,5,6,7,8,9]
    for i in range(9):
        if i==0:
            rename_column(df, str_swap+' Curncy', str_swap)
            continue
        else:
            rename_column(df, f'{str_swap} Curncy.{i}', str_swap)
         
    return df


def rename_column_dates(df, str_column_name, str_new_name):
    time_periods = ['1W', '1M', '1Q', '1Y', '2Y', '5Y', '10Y', '15Y', '20Y']
    
    # Match suffix and assign new name accordingly
    for i in range(1, 9):
        if f'.{i}' in str_column_name:
            new_col_name = str_new_name + '_' + 'dates_'+time_periods[i]
            df.rename(columns={str_column_name: new_col_name}, inplace=True)
            
        else: #renaming the first column as it is recognized as a float, rather #than a string
            df.columns.values[0] = str_new_name + '_dates_' + time_periods[0]
    return df

def call_rename_dates(df, str_new_name):
    for i in range(1, 9):#since the end of the range is exclusive, we start from 1 and end at 9 to include the index 8
        rename_column_dates(df, f'nan.{i}', str_new_name)
    return df

split_dataframes = {}
def split_columns1(df):
    
    for i in range(0, len(df.columns), 2):
        date_col = df.columns[i]
        value_col = df.columns[i + 1]
        # Extract swap and period from value_col, e.g. 'USDJPY_1W'
        try:
            parts = value_col.split('_')
            swap = parts[0]
            period = parts[-1]
            key = f'{swap}_{period}'
            split_dataframes[key] = df[[date_col, value_col]].copy()
        except Exception:
            continue
    return split_dataframes

#using this same implementation for all the other datafreames enclosed wihtin a function
def clean_dataframes_to_date_time_and_numeric(df):
    for column in df.columns:
        if 'dates' in column:
            df[column] = pd.to_datetime(df[column], errors='coerce')
        else:
            df[column] = pd.to_numeric(df[column], errors='coerce')

    return df

#adding a month year column to each dataframe
def add_month_year_column(df):
    for column in df.columns:
        if 'dates' in column.lower():
            df['month'] = df[column].dt.month
            df['year'] = df[column].dt.year
            df['month-year'] = df[column].dt.to_period('M')
    df.drop(columns=['month', 'year'], inplace=True)  # Optionally drop month and year columns if not needed
    return df

def group_by_month_year_mean(df):
    # Group by 'month-year' and calculate the mean for each group
    grouped_df = df.groupby('month-year').mean().reset_index()
    for column in grouped_df.columns:
        if 'dates' in column.lower():
            grouped_df.drop(columns=column, inplace=True)  # Drop date columns if they exist

    return grouped_df
# merging the grouped dataframes into a single dataframe
def merge_grouped_dataframes(dfs):
    merged_df = pd.DataFrame()
    for df in dfs:
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='month-year', how='outer')
    merged_df.dropna(inplace=True)  # Drop rows with any NaN values after merging
    merged_df.reset_index(drop=True, inplace=True)  # Reset the index after merging
    #adding  a base currency column of USD
    merged_df['USD_base'] = [1 for i in range(len(merged_df))]
    return merged_df

# correlation analysis
# step 1 calculate the log returns for each currency based ont the merged dataframe since daily data may be too much noise
def calculate_log_returns(df):
    for column in df.columns:
        if df[column].dtype == float:  # detects if the column is numeric, which means the price
            df['log_returns' + "_"+column] = np.log(df[column] / df[column].shift(1))
            df.dropna(inplace=True)  # Drop rows with any NaN values after calculating log returns
            df.reset_index(drop=True, inplace=True)  # Reset the index after calculating log returns
    return df
log_columns = []

def plot_log_returns(df):
    #change the 'month-year' column to datetime format for better plotting from period format

    
    
    plt.figure(figsize=(12, 6))
    for column in df.columns:
        if 'log_returns' in column:
            log_columns.append(column)
            plt.plot(df['month-year'], df[column], label=column)
    return log_columns


def plot_currency_log_returns(df, currency_cols, date_col='date', log_y=False):
    """
    Plots time series of log returns for each currency in separate subplots.

    Parameters:
        df (DataFrame): Your DataFrame with log returns.
        currency_cols (list): List of column names for currency log returns.
        date_col (str): Name of the column with date/time.
        log_y (bool): If True, sets y-axis to logarithmic scale.
    """
    n = len(currency_cols)
    fig, axs = plt.subplots(n, 1, figsize=(12, 2.5 * n), sharex=True)

    for i, currency in enumerate(currency_cols):
        axs[i].plot(df['month-year'], df[currency], label=currency)
        axs[i].set_title(f'{currency} Log Returns Over Time')
        axs[i].set_ylabel('Log Return')
        axs[i].grid(True)
        if log_y:
            axs[i].set_yscale('symlog')

    axs[-1].set_xlabel('Date')
    plt.tight_layout()
    plt.show()




[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#attribute'

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
